In [ ]:
#Imports 

import numpy as np
import scipy.io as sio

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

import torch
from torchvision import transforms
from PIL import Image

In [ ]:
#Loading the data

data = sio.loadmat('dataset')
x = data['data']
y = data['label']
y = y.reshape(-1,1)
y = np.array(y)
x = x.reshape(51,51,51,5900)
x = np.transpose(x,(3,0,1,2))

# Resizing to be divisible by 14 for DinoV2 Model. 
xx_bin = x[:,0,:42,:42]
xy_bin = x[:,:42,0,:42]
xz_bin = x[:,:42,:42,0]

In [ ]:
transform = transforms.Compose([
                transforms.ToTensor(), 
                 ])


# Making features for specifed model type 
def make_features(x_bin):

    x_t = torch.Tensor()
    for img in x_bin:
        img = Image.fromarray(np.uint8(img*255)).convert('RGB')
        img_t = transform(img)
        img_t = img_t[None,:,:,:]
        x_t = torch.cat((x_t,img_t),0)

    return dinov2_vitb14(x_t).detach().numpy()


# Looping through and creating and saving features for all dino models 
for ver in ['s','b','l','g']:

    dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vit{}14'.format(ver))
    
    
    xx = make_features(xx_bin)
    xy = make_features(xy_bin)
    xz = make_features(xz_bin)
    
    np.save('xx_dino_{}.npy'.format(ver),xx)
    np.save('xy_dino_{}.npy'.format(ver),xy)
    np.save('xz_dino_{}.npy'.format(ver),xz)
    print('Done with xz_dino_{}.npy'.format(ver))